In [26]:
library(ggplot2)
library(ggrepel)
library(RColorBrewer)
library(tidyverse)
library(patchwork)

### functions

In [28]:
get_score_df <- function(genetable_collapsed,score,up_hit='resistance_hit',down_hit='sensitivity_hit',threshold = 4){
    df = genetable_collapsed[,c('gene', 
                                paste0(score,'_ave_Rep1_Rep2_Mann.Whitney.p.value_'),
                                paste0(score,'_ave_Rep1_Rep2_average.phenotype.of.strongest.3_'))
          # which (unlist(gregexpr('rho_ave_Rep1_Rep2', colnames(genetable_collapsed))) == 1))
    ] %>% remove_rownames
    rownames(df) = seq(length=nrow(df))

    colnames(df) <- c('gene','pvalue','score')

    df$score = df$score %>% as.double
    df$pvalue = df$pvalue %>% as.double

    #create dataframe with pseudogene phenotypes
    df$label = '.'

    df %>% mutate(label = replace(label, grepl('pseudo', df[[1]], fixed = TRUE), 'pseudo')) -> df

    pseudo_sd <- df %>% filter(label=='pseudo') %>% select(score) %>% as.list %>% unlist %>% sd

    df %>%
        mutate(label = replace(
            label,score > 0 & label != "pseudo" & score / pseudo_sd * -log10(pvalue) >= threshold,
            up_hit
        ))-> df
    df %>%
        mutate(label = replace(
            label,score < 0 & label != "pseudo" & score / pseudo_sd * -log10(pvalue) <= -threshold,
            down_hit
        )) -> df
    df %>% mutate(label = replace(label, label == '.', 'gene_non_hit')) -> df

    # reorder factors
    df$label <- factor(df$label, levels = c(down_hit, up_hit,'gene_non_hit','pseudo'))
    
    out = list()
    out[["data"]] = df
    out[["pseudo_sd"]] = pseudo_sd
    return(out)
}

# scale_color_manual(values = c('#3182bd', '#de2d26'),
#                    labels = c('Sensitizing Hits', 'Resistance Hits'))

plot_volcano <- function(df, pseudo_sd, score = 'rho',threshold = 4,
                         up_hit='resistance_hit',down_hit='sensitivity_hit',
                         xlim_l=-2.0,xlim_r=2.0,ylim=4.5){
    
    draw_threshold <- function(x){threshold * pseudo_sd * sign(x)/(x)}

    p <- df %>% drop_na %>%
        ggplot(aes(x=score,y=-1*log10(pvalue)) ) + 
        geom_point(
            data = df %>% filter(label=='pseudo'), 
            size = 1, color = 'gray80') +
        geom_point(
            data = df %>% filter(label=='gene_non_hit'), 
            size = 1, color = 'gray90') +
        geom_point(
            data = df %>% filter(label==up_hit), 
            size = 2, color = '#fcae91') + 
        geom_point(
            data = df %>% filter(label==down_hit), 
            size = 2, color = '#bdd7e7') 
    
    if (score == 'gamma'){
        p + xlab(expression('CRISPRi control phenotype (' * gamma * ')')) -> p
    } 
    if (score == 'rho'){
        p + xlab(expression('CRISPRi drug phenotype (' * rho * ')')) -> p
    }
    p +
        theme_classic() +
        xlim(xlim_l,xlim_r) +
        scale_y_continuous(limits = c(0.1,ylim)) +
        ylab(expression('-log'[10] * '(Mann-Whitney p-value)')) +
        stat_function(fun = draw_threshold, linetype = 'dashed', color = 'black') -> p
    
    return(p)
}

label_as_black <- function(p,dd, size = 3, t_x = -0.2, t_y = -0.1){
    p + geom_point(
        data = dd, 
        size = size, shape=21,
        stroke=0.5, 
        colour = "black", fill = "black"
    ) + 
    geom_text_repel(data = dd,
                    aes(label=gene),
                    color = 'black', size = size, nudge_x = t_x, nudge_y = t_y)
}

label_sensitivity_hit <- function(p,dd, size = 2, t_x = -0.2, t_y = -0.1){
    p + geom_point(
        data = dd, 
        size = size, shape=21,
        # stroke=0.5, 
        # colour = "grey30", 
        fill = "#3182bd"
    ) + 
    geom_text_repel(data = dd,
                    aes(label=gene),
                    color = 'black', size = size, nudge_x = t_x, nudge_y = t_y)
}

label_resistance_hit <- function(p,dd, size = 3, t_x = 0.2, t_y = 0.1){
    p + geom_point(
        data = dd, 
        size = size, shape=21,
        stroke=0.5, 
        colour = "grey30", fill = "#de2d26"
    ) + 
    geom_text_repel(data = dd,
                    aes(label=gene),
                    color = 'black', size = size, nudge_x = t_x, nudge_y = t_y)
}

### load ScreenProcessing results and rho $\rho$ scores

In [138]:
read_scores <- function(file_path,threshold){
    genetable_collapsed = read_genetable_collapsed(file_path)

    out = c()

    out$rho = get_score_df(genetable_collapsed,"rho", threshold = threshold)

    out$rho$data = out$rho$data %>% 
        filter(duplicated(gene) == FALSE) %>%
        column_to_rownames('gene')

    out$gamma = get_score_df(genetable_collapsed,"gamma",
                             up_hit = "up_hit", 
                             down_hit = "essential_hit",
                             threshold = threshold)

    out$gamma$data = out$gamma$data %>% 
        filter(duplicated(gene) == FALSE) %>%
        column_to_rownames('gene')
    
    return(out)
}

In [104]:
threshold=1

In [105]:
hl60_exp1_DAC = read_scores('hl60_exp1/genetable_collapsed.txt',threshold=threshold)

In [106]:
molm13_exp_DAC = read_scores('molm13_exp/genetable_collapsed.txt',threshold=threshold)

In [107]:
skm1_exp_DAC = read_scores('skm1_exp/genetable_collapsed.txt',threshold=threshold)

### combined cell lines

In [108]:
cell_lines_DAC = cbind(
    hl60_exp1_DAC$rho$data %>% rename_with( .fn = function(.x){paste0("rho_hl60_", .x)}),
    molm13_exp_DAC$rho$data %>% rename_with( .fn = function(.x){paste0("rho_molm13_", .x)}),
    skm1_exp_DAC$rho$data %>% rename_with( .fn = function(.x){paste0("rho_skm1_", .x)}),

    hl60_exp1_DAC$gamma$data %>% rename_with( .fn = function(.x){paste0("gamma_hl60_", .x)}),
    molm13_exp_DAC$gamma$data %>% rename_with( .fn = function(.x){paste0("gamma_molm13_", .x)}),
    skm1_exp_DAC$gamma$data %>% rename_with( .fn = function(.x){paste0("gamma_skm1_", .x)})
)

In [109]:
cell_lines_DAC %>% group_by(
    # rho_hl60_label,
    # rho_molm13_label,
    # rho_molm13_label,
    gamma_hl60_label,
    # rho_skm1_label
                           ) %>% count

gamma_hl60_label,n
<fct>,<int>
essential_hit,3611
up_hit,1231
gene_non_hit,14063
pseudo,18905


In [110]:
cell_lines_DAC %>% group_by(
    # rho_hl60_label,
    # rho_molm13_label,
    # rho_molm13_label,
    gamma_molm13_label,
    # rho_skm1_label
                           ) %>% count

gamma_molm13_label,n
<fct>,<int>
essential_hit,4199
up_hit,1181
gene_non_hit,13525
pseudo,18905


In [111]:
cell_lines_DAC %>% group_by(
    # rho_hl60_label,
    # rho_molm13_label,
    # rho_molm13_label,
    gamma_skm1_label,
    # rho_skm1_label
                           ) %>% count

gamma_skm1_label,n
<fct>,<int>
essential_hit,4887
up_hit,1470
gene_non_hit,12548
pseudo,18905


In [118]:
bioRxiv_labels = c(
    "DCK",
    "SLC29A1",
    "DCTD", # not in bioRxiv!!
    "DCP2",
    "DCP1A",
    "DCPS",
    "METTL3",
    "KIAA1429",
    "CBLL1",
    "ZC3H13",
    "YTHDF2",
    "YTHDC1",
    "MCL1",
    "BCL2",
    "SQLE",
    "INTS5",
    "MYBBP1A"
)

cell_lines_DAC[bioRxiv_labels,] %>% write.table(
    'cell_lines_DAC_bioRxiv_labels.txt',sep='\t',quote=F,row.names=T
)

In [196]:
cell_lines_DAC %>% write.table(
    'cell_lines_DAC_rho_and_gamma.txt',sep='\t',quote=F,row.names=T
)

In [115]:
# cell_lines_DAC[c('BLM','BTAF1','DNAJA2','ERCC6','ERCC6-PGBD3','FZR1','ILF3','MYBBP1A','PGBD3','PUM1','RAD51','TFRC','WEE1','ZC3H3'),] %>% t

### volcano plots

In [116]:
# # cell_lines_DAC_rho %>% group_by(hl60_label,molm13_label) %>% count
# cell_lines_DAC %>% 
#     # filter(hl60_label=='resistance_hit' & molm13_label=='sensitivity_hit') %>% 
#     # filter(hl60_label=='sensitivity_hit' & molm13_label=='sensitivity_hit') %>% 
#     # filter(hl60_label=='resistance_hit' & molm13_label=='resistance_hit' & skm1_label =='resistance_hit') %>% 
#     filter(rho_hl60_label=='sensitivity_hit' & rho_molm13_label=='sensitivity_hit' & rho_skm1_label =='sensitivity_hit') %>% 
#     arrange(rho_hl60_label) # %>% rownames

### Reference
- ZC3H3 – https://omim.org/entry/618640
    - not very well studied 


- BLM
    - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4052885/


- PGBD3 – PiggyBac Transposable Element Derived 3) 

In [148]:
vol_hl60 = plot_volcano(
    hl60_exp1_DAC$rho$data,hl60_exp1_DAC$rho$pseudo_sd,threshold=threshold, xlim_l = -2,xlim_r = 2,ylim = 4.5    
) %>% label_as_black(
    hl60_exp1_DAC$rho$data[bioRxiv_labels,] %>% rownames_to_column('gene')
) + 
    ggtitle('HL-60') + theme(plot.title = element_text(size = 20,hjust = 0.5) )

In [150]:
res_genes_molm13 = c('DCK',"SLC29A1",'DCTD',"METTL3",'KIAA1429')
sen_genes_molm13 = c('MYBBP1A','INTS5')

vol_molm13 = plot_volcano(
    molm13_exp_DAC$rho$data,molm13_exp_DAC$rho$pseudo_sd,threshold=threshold, xlim_l = -2,xlim_r = 2,ylim = 4.5    
) %>% label_as_black(
    molm13_exp_DAC$rho$data[bioRxiv_labels,] %>% rownames_to_column('gene')
# ) %>% label_resistance_hit(
#     molm13_exp_DAC$rho$data[res_genes_molm13,] %>% rownames_to_column('gene')
# ) %>% label_sensitivity_hit(
#     molm13_exp_DAC$rho$data[sen_genes_molm13,] %>% rownames_to_column('gene')
) + 
    ggtitle('MOLM-13') + theme(plot.title = element_text(size = 20,hjust = 0.5) )

In [151]:
res_genes_skm1 = c('DCK',"SLC29A1",'DCTD',"METTL3",'CBLL1')
sen_genes_skm1 = c("MCL1","BCL2","SQLE",'MYBBP1A',"DCP2","DCPS")

vol_skm1 = plot_volcano(
    skm1_exp_DAC$rho$data,skm1_exp_DAC$rho$pseudo_sd,threshold=threshold, xlim_l = -1.2,xlim_r = 1.2,ylim = 4.5
) %>% label_as_black(
    skm1_exp_DAC$rho$data[bioRxiv_labels,] %>% rownames_to_column('gene')
# ) %>% label_resistance_hit(
#     skm1_exp_DAC$rho$data[res_genes_skm1,] %>% rownames_to_column('gene')
# ) %>% label_sensitivity_hit(
#     skm1_exp_DAC$rho$data[sen_genes_skm1,] %>% rownames_to_column('gene')
) +
    ggtitle('SKM1') + theme(plot.title = element_text(size = 20,hjust = 0.5) )

### gamma vs rho

In [152]:
plot_rho_vs_gamma<-function(df,name){
    df %>% ggplot(aes(x=x, y=y)) + 
        geom_point(
            data = df %>% filter(gamma=='gene_non_hit' & rho=='gene_non_hit'), 
            alpha=10/100, size = 1, color = 'gray90') +
        geom_point(
            data = df %>% filter(gamma=='pseudo'), 
            alpha=1/100, size = 1, color = 'gray90', shape = 21
        ) +
        geom_point(
            data = df %>% filter(rho=='resistance_hit'), 
            alpha=80/100, size = 2, color = '#fcae91') + 
        geom_point(
            data = df %>% filter(rho=='sensitivity_hit'), 
            alpha=80/100, size = 2, color = '#bdd7e7') +
        theme_classic() +
        # xlim(-1.0, 1.0) +
        ylim(-1, 0.19) +
        ylab(expression('CRISPRi control phenotype (' * gamma * ')')) +
        xlab(expression('CRISPRi drug phenotype (' * rho * ')')) +
        theme(axis.text.x = element_text(size = 15, color = 'black'),
              axis.text.y = element_text(size = 15, color = 'black'),
              axis.title.x = element_text(size = 16),
              axis.title.y = element_text(size = 16),
              legend.title = element_blank(),
              legend.text = element_text(size = rel(1.5), color = 'black'),
              legend.position = 'right'
    ) -> pp
    
    pp + ggtitle(name) + theme(
        plot.title = element_text(size = 20,hjust = 0.5)
    ) -> pp
   
    pp + geom_hline(yintercept = 0,linetype = 'dashed', color = 'gray50') -> pp

    return(pp)
}

In [154]:
ann_labels <- function(df, p, res_labels,sen_labels){
    p <- label_sensitivity_hit(p, df %>% rownames_to_column('gene') %>% dplyr::filter(gene %in% sen_labels),size = 3)
    p <- label_resistance_hit(p,  df %>% rownames_to_column('gene') %>% dplyr::filter(gene %in% res_labels),size = 3)
    return(p)
}

In [155]:
# cell_lines_DAC %>% 
#     dplyr::rename(x=rho_hl60_score,y=gamma_hl60_score,rho=rho_hl60_label,gamma=gamma_hl60_label) %>% 
#     select(x,y,rho,gamma) %>%
#     plot_rho_vs_gamma(name='HL-60')

In [160]:
df = cell_lines_DAC %>% 
    dplyr::rename(x=rho_molm13_score,y=gamma_molm13_score,rho=rho_molm13_label,gamma=gamma_molm13_label) %>% 
    select(x,y,rho,gamma) 

rho_gamma_scatter_molm13 = df %>%
    plot_rho_vs_gamma('MOLM-13') %>% 
    label_as_black(df %>% rownames_to_column('gene') %>% dplyr::filter(gene %in% bioRxiv_labels),size = 3)

# rho_gamma_scatter_molm13 = ann_labels(
#     df,rho_gamma_scatter_molm13,
#     res_labels = res_genes_molm13,
#     sen_labels = sen_genes_molm13
# )

In [161]:
df = cell_lines_DAC %>% 
    dplyr::rename(x=rho_skm1_score,y=gamma_skm1_score,rho=rho_skm1_label,gamma=gamma_skm1_label) %>% 
    select(x,y,rho,gamma) 

rho_gamma_scatter_skm1 = df %>%
    plot_rho_vs_gamma('SKM1') %>% 
    label_as_black(df %>% rownames_to_column('gene') %>% dplyr::filter(gene %in% bioRxiv_labels),size = 3)

# rho_gamma_scatter_skm1 = ann_labels(
#     df, rho_gamma_scatter_skm1,
#     res_labels = res_genes_skm1,
#     sen_labels = sen_genes_skm1
# )

### cell line scatter plots

In [162]:
ann_pp <- function(pp,x_limits,y_limits){
    pp + 
        geom_hline(yintercept = 0,linetype = 'dashed', color = 'gray50') +
        geom_vline(xintercept = 0,linetype = 'dashed', color = 'gray50') +
        scale_x_continuous(limits = x_limits,breaks = seq(-1, 1, by = 0.5)) +
        scale_y_continuous(limits = y_limits,breaks = seq(-2, 1, by = 0.5)) -> pp
    
    return(pp)
}

#### rho vs rho

In [181]:
cell_lines_DAC %>% ggplot(aes(x=rho_hl60_score, y=rho_molm13_score)) +
    geom_point(
        data = cell_lines_DAC %>% filter(!(rho_hl60_label=='pseudo' | rho_molm13_label=='pseudo')),
        alpha=10/100, size = 1, color = 'gray10', shape = 21
    ) +
    geom_point(
        data = cell_lines_DAC %>% filter(rho_hl60_label=='pseudo' | rho_molm13_label=='pseudo'),
        alpha=1/100, size = 1, color = 'gray90', shape = 21
    ) +
    geom_point(
        data = cell_lines_DAC %>% filter(rho_hl60_label=='resistance_hit' & rho_molm13_label=='resistance_hit'),
        alpha=30/100, size = 2, color = '#fcae91') + 
    geom_point(
        data = cell_lines_DAC %>% filter(rho_hl60_label=='sensitivity_hit' & rho_molm13_label=='sensitivity_hit'),
        alpha=30/100, size = 2, color = '#bdd7e7') +
    theme_classic() +
#     xlim(-1.0, 1.0) +
#     ylim(-0.7, 0.2) +
    ylab(expression('MOLM-13 CRISPRi drug phenotype (' * rho * ')')) +
    xlab(expression('HL-60 CRISPRi drug phenotype (' * rho * ')')) +
    theme(axis.text.x = element_text(size = 15, color = 'black'),
          axis.text.y = element_text(size = 15, color = 'black'),
          axis.title.x = element_text(size = 16),
          axis.title.y = element_text(size = 16),
          legend.title = element_blank(),
          legend.text = element_text(size = rel(1.5), color = 'black'),
          legend.position = 'right'
) -> pp

pp %>% label_as_black(cell_lines_DAC %>% rownames_to_column('gene') %>% dplyr::filter(gene %in% bioRxiv_labels),size = 3) -> pp

# ann_labels(cell_lines_DAC,pp,
#     res_labels = res_genes_molm13,
#     sen_labels = sen_genes_molm13
# )-> pp

ann_pp(pp,
       c(-1,1),c(-2,1)) -> pp

pp -> rho_hl60_vs_rho_molm13

In [182]:
cell_lines_DAC %>% ggplot(aes(x=rho_hl60_score, y=rho_skm1_score)) +
    geom_point(
        data = cell_lines_DAC %>% filter(!(rho_hl60_label=='pseudo' | rho_skm1_label=='pseudo')),
        alpha=10/100, size = 1, color = 'gray10', shape = 21
    ) +
    geom_point(
        data = cell_lines_DAC %>% filter(rho_hl60_label=='pseudo' | rho_skm1_label=='pseudo'),
        alpha=1/100, size = 1, color = 'gray90', shape = 21
    ) +
    geom_point(
        data = cell_lines_DAC %>% filter(rho_hl60_label=='resistance_hit' & rho_skm1_label=='resistance_hit'),
        alpha=30/100, size = 2, color = '#fcae91') + 
    geom_point(
        data = cell_lines_DAC %>% filter(rho_hl60_label=='sensitivity_hit' & rho_skm1_label=='sensitivity_hit'),
        alpha=30/100, size = 2, color = '#bdd7e7') +
    theme_classic() +
#     xlim(-1.0, 1.0) +
#     ylim(-0.7, 0.2) +
    ylab(expression('SKM-1 CRISPRi drug phenotype (' * rho * ')')) +
    xlab(expression('HL-60 CRISPRi drug phenotype (' * rho * ')')) +
    theme(axis.text.x = element_text(size = 15, color = 'black'),
          axis.text.y = element_text(size = 15, color = 'black'),
          axis.title.x = element_text(size = 16),
          axis.title.y = element_text(size = 16),
          legend.title = element_blank(),
          legend.text = element_text(size = rel(1.5), color = 'black'),
          legend.position = 'right'          
) -> pp

pp %>% label_as_black(cell_lines_DAC %>% rownames_to_column('gene') %>% dplyr::filter(gene %in% bioRxiv_labels),size = 3) -> pp

# ann_labels(cell_lines_DAC,pp,
#     res_labels = res_genes,
#     sen_labels = sen_genes_skm1
# )->pp

ann_pp(pp,
       c(-1,1),c(-1.2,1.2)) -> pp

pp -> rho_hl60_vs_rho_skm1

#### gamma vs gamma

In [189]:
cell_lines_DAC %>% ggplot(aes(x=gamma_hl60_score, y=gamma_molm13_score)) +
    geom_point(
        data = cell_lines_DAC %>% filter(!(gamma_hl60_score=='pseudo' | gamma_molm13_score=='pseudo')),
        alpha=10/100, size = 1, color = 'gray10', shape = 21
    ) +
    geom_point(
        data = cell_lines_DAC %>% filter(gamma_hl60_score=='pseudo' | gamma_molm13_score=='pseudo'),
        alpha=1/100, size = 1, color = 'gray90', shape = 21
    ) +
    geom_point(
        data = cell_lines_DAC %>% filter(gamma_hl60_score=='up_hit' & gamma_molm13_score=='essential_hit'),
        alpha=30/100, size = 2, color = '#fcae91') + 
    geom_point(
        data = cell_lines_DAC %>% filter(gamma_hl60_score=='up_hit' & gamma_molm13_score=='essential_hit'),
        alpha=30/100, size = 2, color = '#bdd7e7') +
    theme_classic() +
    # xlim(-1.0, 1.0) +
    # ylim(-0.7, 0.2) +
    ylab(expression('MOLM-13 CRISPRi control phenotype (' * gamma * ')')) + 
    xlab(expression('HL-60 CRISPRi control phenotype (' * gamma * ')')) +
    theme(axis.text.x = element_text(size = 15, color = 'black'),
          axis.text.y = element_text(size = 15, color = 'black'),
          axis.title.x = element_text(size = 16),
          axis.title.y = element_text(size = 16),
          legend.title = element_blank(),
          legend.text = element_text(size = rel(1.5), color = 'black'),
          legend.position = 'right'
) -> pp

pp %>% label_as_black(cell_lines_DAC %>% rownames_to_column('gene') %>% dplyr::filter(gene %in% bioRxiv_labels),size = 3) -> pp

# ann_labels(cell_lines_DAC,pp,
#     res_labels = res_genes_molm13,
#     sen_labels = sen_genes_molm13
# )-> pp

ann_pp(pp,
       c(-1,0.5),c(-1,0.5)) -> pp

pp -> gamma_hl60_vs_gamma_molm13

In [190]:
cell_lines_DAC %>% ggplot(aes(x=gamma_hl60_score, y=gamma_skm1_score)) +
    geom_point(
        data = cell_lines_DAC %>% filter(!(gamma_hl60_score=='pseudo' | gamma_skm1_score=='pseudo')),
        alpha=10/100, size = 1, color = 'gray10', shape = 21
    ) +
    geom_point(
        data = cell_lines_DAC %>% filter(gamma_hl60_score=='pseudo' | gamma_skm1_score=='pseudo'),
        alpha=1/100, size = 1, color = 'gray90', shape = 21
    ) +
    geom_point(
        data = cell_lines_DAC %>% filter(gamma_hl60_score=='up_hit' & gamma_skm1_score=='essential_hit'),
        alpha=30/100, size = 2, color = '#fcae91') + 
    geom_point(
        data = cell_lines_DAC %>% filter(gamma_hl60_score=='up_hit' & gamma_skm1_score=='essential_hit'),
        alpha=30/100, size = 2, color = '#bdd7e7') +
    theme_classic() +
    # xlim(-1.0, 1.0) +
    # ylim(-0.7, 0.2) +
    ylab(expression('SKM1 CRISPRi control phenotype (' * gamma * ')')) + 
    xlab(expression('HL-60 CRISPRi control phenotype (' * gamma * ')')) +
    theme(axis.text.x = element_text(size = 15, color = 'black'),
          axis.text.y = element_text(size = 15, color = 'black'),
          axis.title.x = element_text(size = 16),
          axis.title.y = element_text(size = 16),
          legend.title = element_blank(),
          legend.text = element_text(size = rel(1.5), color = 'black'),
          legend.position = 'right'
) -> pp

pp %>% label_as_black(cell_lines_DAC %>% rownames_to_column('gene') %>% dplyr::filter(gene %in% bioRxiv_labels),size = 3) -> pp

# ann_labels(cell_lines_DAC,pp,
#     res_labels = res_genes,
#     sen_labels = sen_genes_skm1
# )->pp

ann_pp(pp,
       c(-1,0.5),c(-1,0.5)) -> pp

pp -> gamma_hl60_vs_gamma_skm1

In [191]:
# ggsave('plots/CRISPRi-cell-lines-rho.pdf',
#        ann_pp(pp1,c(-1,1),c(-1.8,1.2)) +
#        ann_pp(pp2,c(-1,1),c(-1.2,1.2)),
#        width = 12, height = 6)

In [192]:
# ggsave('plots/CRISPRi-cell-lines-rho.png',
#        ann_pp(pp1,c(-1,1),c(-1.8,1.2)) +
#        ann_pp(pp2,c(-1,1),c(-1.2,1.2)),
#        width = 12, height = 6)

In [193]:
# Start the PDF device
pdf("multipage_plots.pdf", height=6, width=6)

vol_hl60
vol_molm13
vol_skm1

rho_gamma_scatter_molm13
rho_gamma_scatter_skm1

rho_hl60_vs_rho_molm13
rho_hl60_vs_rho_skm1

gamma_hl60_vs_gamma_molm13
gamma_hl60_vs_gamma_skm1

# End the PDF device
dev.off()

Warning message:
“Removed 3827 rows containing missing values (geom_point).”
Warning message:
“Removed 3495 rows containing missing values (geom_point).”
Warning message:
“Removed 72 row(s) containing missing values (geom_path).”
Warning message:
“Removed 3567 rows containing missing values (geom_point).”
Warning message:
“Removed 3500 rows containing missing values (geom_point).”
Warning message:
“Removed 1 rows containing missing values (geom_point).”
Warning message:
“Removed 36 row(s) containing missing values (geom_path).”
Warning message:
“Removed 3509 rows containing missing values (geom_point).”
Warning message:
“Removed 3043 rows containing missing values (geom_point).”
Warning message:
“Removed 62 row(s) containing missing values (geom_path).”
Warning message:
“Removed 1 rows containing missing values (geom_point).”
Warning message:
“Removed 1 rows containing missing values (geom_text_repel).”
Warning message:
“Removed 10 rows containing missing values (geom_point).”
Warning 

png 
  2

___

In [90]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /data_gilbert/home/aarab/anaconda3/envs/deseq2/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] patchwork_1.1.1    forcats_0.5.1      stringr_1.4.0      dplyr_1.0.7       
 [5] purrr_0.3.4        readr_2.0.2        tidyr_1.1.4        tibble_3.1.6      
 [9] tidyverse_1.3.1    RColorBrewer_1.1-2 ggrepel_0.9.1      ggplot2_3.3.5     

loaded via a namespace (and not attached):
 [1